#PEFT

PEFT, acronimo di Parameter-Efficient Fine-Tuning, è una tecnica utilizzata nell'ambito del deep learning per adattare in modo efficiente i modelli di lingua su dataset specifici senza dover riaddestrare l'intero modello. Invece di aggiornare tutti i parametri del modello durante il fine-tuning, PEFT identifica e aggiorna solo una piccola parte dei parametri, riducendo così il carico computazionale e la quantità di dati necessari per il fine-tuning.

L'obiettivo principale di PEFT è rendere il processo di adattamento dei modelli di lingua più efficiente in termini di risorse computazionali e di dati. Questo è particolarmente importante quando si lavora con modelli di lingua molto grandi e complessi, dove il fine-tuning completo potrebbe richiedere molte risorse e tempo.

PEFT può essere implementato in vari modi, ad esempio tramite l'aggiunta di piccoli adattatori o strati di fine-tuning a modelli preaddestrati, oppure attraverso tecniche di compressione e quantizzazione dei parametri. L'obiettivo principale è ottenere un buon bilanciamento tra le prestazioni del modello e l'efficienza delle risorse durante il fine-tuning.

##Configurazione

In [ ]:
!pip install transformers peft evaluate datasets

In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from sklearn.model_selection import train_test_split
import evaluate
import torch
import numpy as np
import pandas as pd

##Caricamento e manipolazione del dataset

In [4]:
# Carica il dataset
dataset = load_dataset("AbdulHadi806/mail_spam_ham_dataset")

# Divisione dataset in train e validation set
train_dataset, validation_dataset = train_test_split(dataset["train"], test_size=0.2, shuffle=True)

# Manipolazione dataset
train_dataset = Dataset.from_dict(train_dataset)
validation_dataset = Dataset.from_dict(validation_dataset)

# Rinominazione 'Category' in 'label'
train_dataset = train_dataset.rename_column("Category", "label")
validation_dataset = validation_dataset.rename_column("Category", "label")

# Trasformazione delle etichette da 'ham'/'spam' a 0/1
train_dataset = train_dataset.map(lambda example: {'label': 0 if example['label'] == 'ham' else 1})
validation_dataset = validation_dataset.map(lambda example: {'label': 0 if example['label'] == 'ham' else 1})

# Creazione oggetto DatasetDict contenente i dataset di train e validation
dataset_dict = DatasetDict({"train": train_dataset, "validation": validation_dataset})

# Stampa dataset_dict
print(dataset_dict)

# Stampa nuove etichette
print(train_dataset['label'])

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

Map:   0%|          | 0/1123 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'Message'],
        num_rows: 4490
    })
    validation: Dataset({
        features: ['label', 'Message'],
        num_rows: 1123
    })
})
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 

##Modello

In [5]:
# Definizione del modello
model_checkpoint = 'bert-base-uncased'

# Definizione del mapping delle labels
id2label = {0: "SPAM", 1: "HAM"}
label2id = {"SPAM":0, "HAM":1}

# Generazione del modello di classificazione dal modello base
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Creazione tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# Aggiunta pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Definizione funzione di tokenizzazione
def tokenize_function(examples):
    text = examples["Message"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [8]:
# Tokenizzazione training e validation datasets
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

Map:   0%|          | 0/1123 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'Message', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4490
    })
    validation: Dataset({
        features: ['label', 'Message', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1123
    })
})

In [9]:
# Definizione data_collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# Importa la misura di valutazione > accuracy
accuracy = evaluate.load("accuracy")

In [11]:
# Definizione funzione di valutazione
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
# Configurazione per adattare il modello usando qlora
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01
)

In [15]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 148,994 || all params: 109,632,772 || trainable%: 0.13590279373762437


In [16]:
# Configurazione iperparametri
lr = 1e-3
batch_size = 4
num_epochs = 5

In [17]:
# Configurazione dei training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# Creazione trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

##Addestramento e valutazione

In [18]:
# Addestramento del modello e valutazione
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.125200,0.141076,{'accuracy': 0.9830810329474622}


Trainer is attempting to log a value of "{'accuracy': 0.9830810329474622}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.125200,0.141076,{'accuracy': 0.9830810329474622}
2,0.459200,0.536357,{'accuracy': 0.8619768477292965}
3,0.169900,0.102577,{'accuracy': 0.981300089047195}
4,0.121900,0.164378,{'accuracy': 0.968833481745325}
5,0.087800,0.079181,{'accuracy': 0.9857524487978628}


Trainer is attempting to log a value of "{'accuracy': 0.8619768477292965}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.981300089047195}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.968833481745325}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9857524487978628}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5615, training_loss=0.16806563065716548, metrics={'train_runtime': 304.3158, 'train_samples_per_second': 73.772, 'train_steps_per_second': 18.451, 'total_flos': 503268083582208.0, 'train_loss': 0.16806563065716548, 'epoch': 5.0})